In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv("KS_train_data.csv", delimiter=',') 

In [7]:
data.head(10)

,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,...,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url
0,KS_000000,80,I will be an artist-in-residence at Elsewhere ...,art,3596,US,1325980505,USD,1334866560,1.0,...,1332969260,Elsewhere Studios artist-in-residency program!,3596.00,False,3596.00,"Paonia, CO",True,mixed media,https://www.kickstarter.com/projects/hilaryeme...,https://www.kickstarter.com/projects/hilaryeme...
1,KS_000001,82,1000 Artists is a public art-making installati...,art,4586,US,1330926084,USD,1336440145,1.0,...,1332984145,1000 Artists: Presidential Inauguration 2013,4586.00,False,4586.00,"Washington, DC",True,public art,https://www.kickstarter.com/projects/17146650/...,https://www.kickstarter.com/projects/17146650/...
2,KS_000002,21,"The Sequel to ‘My Favorite Machine”, ""MyPhoneH...",art,5217,US,1332382894,USD,1338175739,1.0,...,1332991739,MyPhoneHenge,5217.00,False,5217.00,"Frisco, TX",True,sculpture,https://www.kickstarter.com/projects/belveal/m...,https://www.kickstarter.com/projects/belveal/m...
3,KS_000003,37,A film exploring the role and value of art edu...,art,7160,GB,1332342451,USD,1334806096,1.0,...,1332991696,Walk-Through,7160.00,False,7160.00,"Glasgow, UK",True,art,https://www.kickstarter.com/projects/107813091...,https://www.kickstarter.com/projects/107813091...
4,KS_000004,153,We need to build a kitchen for Habitable Space...,art,15445,US,1328562617,USD,1335584240,1.0,...,1332992240,A kitchen for Habitable Spaces,15445.00,False,15445.00,"Kingsbury, TX",True,public art,https://www.kickstarter.com/projects/104409276...,https://www.kickstarter.com/projects/104409276...
5,KS_000005,18,Source Material is 2 person exhibition of the ...,art,2190,US,1331239703,USD,1336090320,1.0,...,1333017856,Source Material: Works by Brian Dupont and Chr...,2190.00,False,2190.00,"Houston, TX",True,painting,https://www.kickstarter.com/projects/973860748...,https://www.kickstarter.com/projects/973860748...
6,KS_000006,156,An anthropological field study of the Dwarven ...,art,5429,US,1332802530,USD,1334030340,1.0,...,1333045464,A Book of Dwarves,5429.95,False,5429.95,"Chicago, IL",True,art,https://www.kickstarter.com/projects/136757708...,https://www.kickstarter.com/projects/136757708...
7,KS_000007,479,Portrait artist seeks funds to kickstart an ex...,art,62736,US,1331325438,USD,1335671940,1.0,...,1332822807,Drawing a Drawing 365,62736.01,True,62736.01,"New Britain, CT",True,painting,https://www.kickstarter.com/projects/michaelsh...,https://www.kickstarter.com/projects/michaelsh...
8,KS_000008,41,This is a month long Residency at the The Verm...,art,2930,US,1331904129,USD,1336736610,1.0,...,1332848610,The Vermont Studio Center Artist Residency Pro...,2930.00,True,2930.00,"Johnson, VT",True,painting,https://www.kickstarter.com/projects/106325273...,https://www.kickstarter.com/projects/106325273...
9,KS_000009,25,"Art, Music, and Theater live on the Waterfront...",art,5001,US,1330979179,USD,1335443088,1.0,...,1332851088,Arts on the Waterfront,5001.00,False,5001.00,"Seattle, WA",True,public art,https://www.kickstarter.com/projects/139851890...,https://www.kickstarter.com/projects/139851890...


In [6]:
data['converted_pledged_amount'].describe()

count    1.000000e+05
mean     1.241108e+04
std      1.005545e+05
min      0.000000e+00
25%      1.250000e+02
50%      1.650000e+03
75%      6.271250e+03
max      1.026684e+07
Name: converted_pledged_amount, dtype: float64

In [18]:
#here 2 possible outliers can be seen at the top
data.nlargest(15,['converted_pledged_amount'] )

,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,...,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url
19474,KS_019474,68929,Pebble is a customizable watch. Download new w...,design,10266845,US,1332451302,USD,1337396400,1.0,...,1334120344,Pebble: E-Paper Watch for iPhone and Android,10266845.74,True,10266845.74,"Palo Alto, CA",True,product design,https://www.kickstarter.com/projects/getpebble...,https://www.kickstarter.com/projects/getpebble...
23683,KS_023683,68929,Pebble is a customizable watch. Download new w...,design,10266845,US,1332451302,USD,1337396400,1.0,...,1334120344,Pebble: E-Paper Watch for iPhone and Android,10266845.74,True,10266845.74,"Palo Alto, CA",True,product design,https://www.kickstarter.com/projects/getpebble...,https://www.kickstarter.com/projects/getpebble...
50400,KS_050400,63416,Cracking open the last closed platform: the TV...,games,8596474,US,1337044803,USD,1344488400,1.0,...,1341924281,OUYA: A New Kind of Video Game Console,8596474.58,True,8596474.58,"Los Angeles, CA",True,gaming hardware,https://www.kickstarter.com/projects/ouya/ouya...,https://www.kickstarter.com/projects/ouya/ouya...
52865,KS_052865,69320,Yu Suzuki presents the long awaited third inst...,games,6333295,JP,1433238208,USD,1437184264,1.0,...,1434419464,Shenmue 3,6333295.77,True,6333295.77,"Tokyo, Japan",True,video games,https://www.kickstarter.com/projects/ysnet/she...,https://www.kickstarter.com/projects/ysnet/she...
86354,KS_086354,18220,Pono's mission is to provide the best possible...,technology,6225354,US,1393347728,USD,1397583042,1.0,...,1394559042,Pono Music - Where Your Soul Rediscovers Music,6225354.98,True,6225354.98,"San Francisco, CA",True,sound,https://www.kickstarter.com/projects/100361482...,https://www.kickstarter.com/projects/100361482...
95028,KS_095028,18220,Pono's mission is to provide the best possible...,technology,6225354,US,1393347728,USD,1397583042,1.0,...,1394559042,Pono Music - Where Your Soul Rediscovers Music,6225354.98,True,6225354.98,"San Francisco, CA",True,sound,https://www.kickstarter.com/projects/100361482...,https://www.kickstarter.com/projects/100361482...
36247,KS_036247,91585,UPDATED: This is it. We're making a Veronica M...,film & video,5702153,US,1320444480,USD,1365822000,1.0,...,1363185742,The Veronica Mars Movie Project,5702153.38,True,5702153.38,"San Diego, CA",True,narrative film,https://www.kickstarter.com/projects/559914737...,https://www.kickstarter.com/projects/559914737...
86254,KS_086254,105857,Bring Reading Rainbow’s library of interactive...,technology,5408916,US,1398222021,USD,1404327600,1.0,...,1401282345,"Bring Reading Rainbow Back for Every Child, Ev...",5408916.95,True,5408916.95,"Los Angeles, CA",True,web,https://www.kickstarter.com/projects/readingra...,https://www.kickstarter.com/projects/readingra...
98043,KS_098043,105857,Bring Reading Rainbow’s library of interactive...,technology,5408916,US,1398222021,USD,1404327600,1.0,...,1401282345,"Bring Reading Rainbow Back for Every Child, Ev...",5408916.95,True,5408916.95,"Los Angeles, CA",True,web,https://www.kickstarter.com/projects/readingra...,https://www.kickstarter.com/projects/readingra...
48966,KS_048966,74405,A story-driven CRPG set in the world of Monte ...,games,4188927,US,1359134981,USD,1365206400,1.0,...,1362577657,Torment: Tides of Numenera,4188927.49,True,4188927.49,"Newport Beach, CA",True,video games,https://www.kickstarter.com/projects/inxile/to...,https://www.kickstarter.com/projects/inxile/to...
